In [1]:
import pandas as pd
import json
import os
from collections import Counter
import re
from dateutil.parser import parse
import xmltodict
import yaml
import sys
import glob
import ast
from datetime import datetime
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import gzip

In [2]:
path='/HDD16TB/weisi/qa_Health_and_Personal_Care.json.gz'
multipath='/HDD16TB/weisi/QA_Health_and_Personal_Care.json.gz'
metapath='/HDD16TB/weisi/meta_Health_and_Personal_Care.json.gz'


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(path)
df.head()


,questionType,asin,answerTime,unixTime,question,answer,answerType
0,open-ended,7293000154,"Dec 20, 2013",1.387526e+09,which model Norelcos does it fit?,"here are 3 norelco shavers this fits: 715 RL, ...",NaN
1,open-ended,7293000154,"May 27, 2014",1.401174e+09,What replacement blades do I use for my 5801XL...,The best I can do from the info from Norelco i...,NaN
2,yes/no,7293000154,"Jun 7, 2014",1.402124e+09,are these made by philips norelco?,Yes and they work great.each set last me one y...,Y
3,open-ended,7293000154,"Apr 15, 2014",1.397545e+09,I have a philips HQ 6695 does it fit?,yes,NaN
4,open-ended,7293000154,"Mar 22, 2014",1.395472e+09,I have a pt 730 and I would like to get replac...,"I have purchased replacement blades, it's not ...",NaN


In [3]:
metadf=getDF(metapath)
metadf.head()

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN
3,0615315860,NaN,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],NaN,NaN
4,0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],NaN,NaN


In [4]:
multidf = getDF(multipath)
multidf.head()

,asin,questions
0,B000052XUV,"[{'questionType': 'open-ended', 'askerID': 'A1..."
1,B00005B6ZL,"[{'questionType': 'open-ended', 'askerID': 'AH..."
2,B00005M1WE,"[{'questionType': 'open-ended', 'askerID': 'A3..."
3,B00005NAUI,"[{'questionType': 'open-ended', 'askerID': 'AW..."
4,B00005R19P,"[{'questionType': 'open-ended', 'askerID': 'A2..."


In [7]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

metadata=parse(metapath) #dictionary

count = 0
for item in metadata:
    print(item)
    count += 1
    if count == 10:
        break

{'asin': '0077614992', 'description': 'This is an example product description.', 'title': 'Principles of Mgmt + Oper-CSUF Custom C', 'imUrl': 'http://ecx.images-amazon.com/images/I/51G%2BRqOCiqL._SY300_.jpg', 'related': {'also_bought': ['0471730726', '0132834871', '0471391905', 'B00000JZKB', '0324314132', '0073525057', '1133227295'], 'also_viewed': ['0073525057', '1133227295', '0324628676', '0073523224']}, 'salesRank': {'Health & Personal Care': 168429}, 'categories': [['Health & Personal Care']]}
{'asin': '0615208479', 'description': "By now we all know the benefits of exercise for the body. It's the only real fountain of youth! The same is true for the brain. Take your brain to the gym several times a week and you can improve, regain and prevent memory loss. Discover the world of brain fitness through BrainAerboics.\nThe program was designed by a medical team and is backed with mounting research proving it works. It is believed to be the only one that combines the three crucial eleme

In [4]:
metadf.iloc[0]#asin,description,title,salesRank,price,brand

asin                                                  0077614992
description              This is an example product description.
title                    Principles of Mgmt + Oper-CSUF Custom C
imUrl          http://ecx.images-amazon.com/images/I/51G%2BRq...
related        {'also_bought': ['0471730726', '0132834871', '...
salesRank                     {'Health & Personal Care': 168429}
categories                            [[Health & Personal Care]]
price                                                        NaN
brand                                                        NaN
Name: 0, dtype: object

In [4]:
metadf.fillna('x', inplace=True) #fills missing (NaN) values in the with the character 'x'
metadf.head()

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],x,x
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,x,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",x,x
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,x
3,0615315860,x,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],x,x
4,0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],x,x


In [6]:
tok = RegexpTokenizer(r'\w+')  # alphanumeric tokenization

def preprocess(doc, stopwords=None):
    global tok
    
    # If the 'stopwords' argument is not provided, set it to an empty set.
    if stopwords is None:
        stopwords = set()

    # replace URLs in the doc string with the word "url"
    doc = re.sub(r"https?:\S+", "url", doc)
    # remove newlines and tabs 
    doc = doc.replace('\n', ' ')
    doc = doc.replace('\t', ' ')
    
    # replace date
    doc = re.sub(r"(\d+)+(\-)+(\d+)+(\-)+(\d+)", "date", doc)
    # remove all serialization eg 1. 1) or 1.1
    doc = re.sub(r"(\d+)+(\.|\))+(\d+)", "", doc)
    doc = re.sub(r"(\d+)+(\.|\))", "", doc)

    doc = re.sub(r"\b(\w+)( \1\b)+", r"\1", doc)  # removing consecutive duplicate words
    doc = re.sub(r"([^A-Za-z0-9\s](\s)){2,}", " ", doc)  # remove consecutive punctuations

    
    doc = re.sub(r'\.+', '..', doc) #replaces two or more consecutive ellipsis with just two (..).
    doc = re.sub(r'!+', '!', doc) #replaces two or more consecutive ! with just one 
    doc = re.sub(r'\*+', ' ', doc) #replaces two or more consecutive * with just space
    doc = re.sub(r'_+', ' ', doc) #replaces two or more consecutive underscore_
    doc = re.sub(r',+', ',', doc) #replaces two or more consecutive commas,

    # all lowercase
    doc = doc.lower()

    doc = [item.strip() for item in tok.tokenize(doc)
           if len(item.strip()) > 1 and item not in stopwords
           ]  # tokenize

    return doc

In [7]:
bidf = df[df['questionType'].isin(['yes/no'])]
bidf.head()

,questionType,asin,answerTime,unixTime,question,answer,answerType
2,yes/no,7293000154,"Jun 7, 2014",1.402124e+09,are these made by philips norelco?,Yes and they work great.each set last me one y...,Y
6,yes/no,7884890364,"Oct 31, 2013",1.383203e+09,Is there a sell-by date on these packages?,There is no mfg date or expiry date on these b...,N
7,yes/no,7884890364,"Feb 21, 2015",1.424506e+09,Are they the same as AG 13?,Yes,Y
8,yes/no,7884890364,"Jan 24, 2015",1.422086e+09,Are these the same as the LR44GD,i do believe they are.,?
10,yes/no,7884890364,"Dec 24, 2013",1.387872e+09,Do they have the New Hologram Package ?,Orange and black package. Checked all the volt...,?


In [10]:
bi_null = bidf['answerTime'].isnull().any()
print(bi_null)
df_null = df['answerTime'].isnull().any()
print(df_null)

bi_unix_null = bidf['unixTime'].isnull().any()
print(bi_unix_null)
df_unix_null = df['unixTime'].isnull().any()
print(df_unix_null)
#unixTime has some nulls 
print(f"Number of rows in bidf: {bidf.shape[0]}")
print(f"Number of rows in metadf: {metadf.shape[0]}")


False
False
True
True
Number of rows in bidf: 33433
Number of rows in metadf: 263032


In [8]:
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
bidf.question = df.question.progress_apply(lambda x: preprocess(x))
bidf.answer  = df.answer .progress_apply(lambda x: preprocess(x))
#join the list of tokens back into a single string with space-separated words.
df.question = df.question.apply(lambda x: ' '.join(x))
df.answer = df.answer.apply(lambda x: ' '.join(x))

100%|██████████| 80496/80496 [00:01<00:00, 65535.69it/s]
/tmp/ipykernel_1553239/4144279059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidf.question = df.question.progress_apply(lambda x: preprocess(x))
100%|██████████| 80496/80496 [00:48<00:00, 1666.72it/s] 
/tmp/ipykernel_1553239/4144279059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidf.answer  = df.answer .progress_apply(lambda x: preprocess(x))


In [9]:
bidf['answerTime'] = pd.to_datetime(bidf['unixTime'], unit='s').dt.strftime('%Y-%m-%d')  #UnixTime has null val
bidf_cleaned = bidf.dropna(subset=['answerTime'])

#bidf['answerTime'] = bidf['answerTime'].dt.strftime('%Y-%m-%d')
'''def convert_to_iso(date_str):
    date_obj = datetime.strptime(date_str, '%b %d, %Y')
    return date_obj.strftime('%Y-%m-%d')

bidf['answerTime'] = bidf['answerTime'].apply(convert_to_iso)
#time data '10 days ago' does not match format '%b %d, %Y'''
bidf.head()


/tmp/ipykernel_1553239/312538677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidf['answerTime'] = pd.to_datetime(bidf['unixTime'], unit='s').dt.strftime('%Y-%m-%d')  #UnixTime has null val


,questionType,asin,answerTime,unixTime,question,answer,answerType
2,yes/no,7293000154,2014-06-07,1.402124e+09,"[are, these, made, by, philips, norelco]","[yes, and, they, work, great, each, set, last,...",Y
6,yes/no,7884890364,2013-10-31,1.383203e+09,"[is, there, sell, by, date, on, these, packages]","[there, is, no, mfg, date, or, expiry, date, o...",N
7,yes/no,7884890364,2015-02-21,1.424506e+09,"[are, they, the, same, as, ag, 13]",[yes],Y
8,yes/no,7884890364,2015-01-24,1.422086e+09,"[are, these, the, same, as, the, lr44gd]","[do, believe, they, are]",?
10,yes/no,7884890364,2013-12-24,1.387872e+09,"[do, they, have, the, new, hologram, package]","[orange, and, black, package, checked, all, th...",?


In [10]:
tqdm.pandas()#sets up the tqdm progress bar for tracking the progress of the following operation
metadf.description= metadf.description.progress_apply(lambda x: preprocess(x))
#metadf.description = metadf.description.apply(lambda x: ' '.join(x))


 89%|████████▉ | 235303/263032 [00:52<00:06, 4487.98it/s] 


KeyboardInterrupt: 

In [11]:
tqdm.pandas()
metadf.title  = metadf.title .progress_apply(lambda x: preprocess(x))
#metadf.title = metadf.title.apply(lambda x: ' '.join(x))
metadf.head()

100%|██████████| 263032/263032 [00:03<00:00, 66423.71it/s]


,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,"[principles, of, mgmt, oper, csuf, custom]",http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],x,x
1,0615208479,By now we all know the benefits of exercise fo...,"[brain, fitness, exercises, software]",http://ecx.images-amazon.com/images/I/41kbZB04...,x,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",x,x
2,0615269990,What's wrong with your patient?Do all the symp...,"[occam, razor]",http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,x
3,0615315860,x,"[101, blenderbottle, recipes, quick, and, easy]",http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],x,x
4,0615406394,This is an example product description.,"[aphrodite, reborn, women, stories, of, hope, ...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],x,x


In [ ]:
'''# convert to a dictionary: 
metadf_dict = dict((z[0], list(z[1:])) for z in zip(
    metadf.asin, metadf['description'], metadf['title'], 
    metadf['salesRank'], metadf['price'], metadf['brand'], metadf['categories']
))'''

#salesRank,price,brand,categories

In [12]:
# Merge the dataframes on the 'asin' column using a left merge
merged_df = pd.merge(bidf_cleaned, metadf[['asin', 'description', 'title', 'salesRank', 'price', 'brand']], on='asin', how='left')

# Export the merged dataframe to a JSON file
merged_df.to_json('combined_data.json', orient='records', lines=True)

In [13]:
combined = pd.read_json('combined_data.json', lines=True)
combined.head()

#33433 lines some of the lines: "answerTime":null,"unixTime":null, (10)

,questionType,asin,answerTime,unixTime,question,answer,answerType,description,title,salesRank,price,brand
0,yes/no,7293000154,2014-06-07,1402124400,"[are, these, made, by, philips, norelco]","[yes, and, they, work, great, each, set, last,...",Y,For best results replace shaver head every 2 y...,"[philips, norelco, hq56, 52, reflex, plus, and...",{'Health & Personal Care': 77771},24.86,Philips
1,yes/no,7884890364,2013-10-31,1383202800,"[is, there, sell, by, date, on, these, packages]","[there, is, no, mfg, date, or, expiry, date, o...",N,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell
2,yes/no,7884890364,2015-02-21,1424505600,"[are, they, the, same, as, ag, 13]",[yes],Y,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell
3,yes/no,7884890364,2015-01-24,1422086400,"[are, these, the, same, as, the, lr44gd]","[do, believe, they, are]",?,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell
4,yes/no,7884890364,2013-12-24,1387872000,"[do, they, have, the, new, hologram, package]","[orange, and, black, package, checked, all, th...",?,Maxell LR44 Batteries 10 Pack,"[maxell, lr44, batteries, 10, pack]",x,0.92,Maxell


In [17]:
bidf.iloc[9]  #delete data or keep it as blank?

questionType                                               yes/no
asin                                                   B00000J47L
answerTime                                                    NaN
unixTime                                                      NaN
question        [can, charge, these, batteries, 2300mah, with,...
answer                                 [you, can, mix, batteries]
answerType                                                      ?
Name: 21, dtype: object

In [20]:
has_null = df['answerTime'].isnull().any()
print(has_null)

has_null = bidf['answerTime'].isnull().any()
print(has_null)


False
True


-----------------------------------------------------------------explore----------------------------------------------------

In [32]:
print(df.iloc[535])
print('answer\n',df.at[535,'answer'],'\n')

print(df.iloc[1])
print('answer\n',df.at[1,'answer'],'\n')
print('answerType\n',df.at[1,'answerType'],'\n')


questionType                                               yes/no
asin                                                   B00006IA93
answerTime                                           Nov 18, 2013
unixTime                                             1384761600.0
question             can I use scotchgard fabric on a braided rug
answer          i cant imagine why you couldnt. just be sure y...
answerType                                                      Y
Name: 535, dtype: object
answer
 i cant imagine why you couldnt. just be sure you get into the little squiggle places of the braids. should work fine. when the rug is cleaned, it will wash out and need to be redone. 

questionType                                           open-ended
asin                                                   7293000154
answerTime                                           May 27, 2014
unixTime                                             1401174000.0
question        What replacement blades do I use for my 

In [33]:
print(df.iloc[3]) #seems that if the question is open-ended the answertype would be NaN even it's a yes


questionType                               open-ended
asin                                       7293000154
answerTime                               Apr 15, 2014
unixTime                                 1397545200.0
question        I have a philips HQ 6695 does it fit?
answer                                            yes
answerType                                        NaN
Name: 3, dtype: object


In [34]:
print(df.iloc[8])
print('answer\n',df.at[8,'answer'],'\n')

print(df.iloc[10])
print('answer\n',df.at[10,'answer'],'\n')

questionType                              yes/no
asin                                  7884890364
answerTime                          Jan 24, 2015
unixTime                            1422086400.0
question        Are these the same as the LR44GD
answer                    i do believe they are.
answerType                                     ?
Name: 8, dtype: object
answer
 i do believe they are. 

questionType                                               yes/no
asin                                                   7884890364
answerTime                                           Dec 24, 2013
unixTime                                             1387872000.0
question                  Do they have the New Hologram Package ?
answer          Orange and black package. Checked all the volt...
answerType                                                      ?
Name: 10, dtype: object
answer
 Orange and black package. Checked all the voltages. Checked out. 



In [18]:
df['answerTime'].astype(str).unique()

array(['Dec 20, 2013', 'May 27, 2014', 'Jun 7, 2014', ..., 'Aug 7, 2012',
       'Apr 24, 2015', 'May 2, 2015'], dtype=object)

In [36]:
df['answerType'].astype(str).unique()

array(['nan', 'Y', 'N', '?'], dtype=object)

In [37]:
df['questionType'].astype(str).unique()

array(['open-ended', 'yes/no'], dtype=object)

In [38]:
sorted(df['unixTime'].astype(str).unique())
 

['1143187200.0',
 '1143705600.0',
 '1144738800.0',
 '1146726000.0',
 '1147849200.0',
 '1149058800.0',
 '1150873200.0',
 '1151650800.0',
 '1151737200.0',
 '1154070000.0',
 '1154588400.0',
 '1154934000.0',
 '1158735600.0',
 '1159254000.0',
 '1161846000.0',
 '1163232000.0',
 '1164009600.0',
 '1164182400.0',
 '1165478400.0',
 '1165737600.0',
 '1165996800.0',
 '1167379200.0',
 '1168848000.0',
 '1169280000.0',
 '1171267200.0',
 '1171699200.0',
 '1171785600.0',
 '1171872000.0',
 '1172304000.0',
 '1172476800.0',
 '1172736000.0',
 '1173254400.0',
 '1173600000.0',
 '1173769200.0',
 '1174028400.0',
 '1174719600.0',
 '1176015600.0',
 '1176361200.0',
 '1179817200.0',
 '1179903600.0',
 '1180076400.0',
 '1180940400.0',
 '1182236400.0',
 '1182668400.0',
 '1183791600.0',
 '1186470000.0',
 '1187766000.0',
 '1188889200.0',
 '1189062000.0',
 '1190185200.0',
 '1191481200.0',
 '1192172400.0',
 '1192258800.0',
 '1192950000.0',
 '1193122800.0',
 '1193209200.0',
 '1193814000.0',
 '1193986800.0',
 '1194422400.0

In [39]:
print(datetime.fromtimestamp(1394179200.0))
print(datetime.fromtimestamp(1143187200.0))


2014-03-07 02:00:00
2006-03-24 02:00:00


In [40]:
bidf = df[df['questionType'].isin(['yes/no'])]
bidf.head

<bound method NDFrame.head of       questionType        asin    answerTime      unixTime  \
2           yes/no  7293000154   Jun 7, 2014  1.402124e+09   
6           yes/no  7884890364  Oct 31, 2013  1.383203e+09   
7           yes/no  7884890364  Feb 21, 2015  1.424506e+09   
8           yes/no  7884890364  Jan 24, 2015  1.422086e+09   
10          yes/no  7884890364  Dec 24, 2013  1.387872e+09   
...            ...         ...           ...           ...   
80485       yes/no  B00L3H2NHM   Oct 5, 2014  1.412492e+09   
80489       yes/no  B00L9NR672  Jul 30, 2014  1.406704e+09   
80492       yes/no  B00L9NR672   Aug 6, 2014  1.407308e+09   
80493       yes/no  B00L9NR672  Jul 19, 2014  1.405753e+09   
80495       yes/no  B00L9NR672  Apr 20, 2015  1.429513e+09   

                                                question  \
2                     are these made by philips norelco?   
6             Is there a sell-by date on these packages?   
7                            Are they the sam

In [41]:
bidf['answerType'].astype(str).unique()

array(['Y', 'N', '?'], dtype=object)

In [42]:
YorNdf = df[df['answerType'].isin(['Y','N'])]
YorNdf.head

<bound method NDFrame.head of       questionType        asin    answerTime      unixTime  \
2           yes/no  7293000154   Jun 7, 2014  1.402124e+09   
6           yes/no  7884890364  Oct 31, 2013  1.383203e+09   
7           yes/no  7884890364  Feb 21, 2015  1.424506e+09   
15          yes/no  B00000J47L  Jan 27, 2014  1.390810e+09   
23          yes/no  B00000J47L  Feb 21, 2015  1.424506e+09   
...            ...         ...           ...           ...   
80478       yes/no  B00L3H2NHM  Jul 16, 2014  1.405494e+09   
80480       yes/no  B00L3H2NHM   Jul 6, 2014  1.404630e+09   
80484       yes/no  B00L3H2NHM  Mar 28, 2015  1.427526e+09   
80485       yes/no  B00L3H2NHM   Oct 5, 2014  1.412492e+09   
80492       yes/no  B00L9NR672   Aug 6, 2014  1.407308e+09   

                                                question  \
2                     are these made by philips norelco?   
6             Is there a sell-by date on these packages?   
7                            Are they the sam

In [4]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

alldata=parse(path)

count = 0
for item in alldata:
    print(item)
    count += 1
    if count == 10:
        break

{'questionType': 'open-ended', 'asin': '7293000154', 'answerTime': 'Dec 20, 2013', 'unixTime': 1387526400, 'question': 'which model Norelcos does it fit?', 'answer': 'here are 3 norelco shavers this fits: 715 RL, 725 RL, 6940LC'}
{'questionType': 'open-ended', 'asin': '7293000154', 'answerTime': 'May 27, 2014', 'unixTime': 1401174000, 'question': 'What replacement blades do I use for my 5801XL razor? It was a GQ9', 'answer': 'The best I can do from the info from Norelco is an HQ9.'}
{'questionType': 'yes/no', 'asin': '7293000154', 'answerTime': 'Jun 7, 2014', 'unixTime': 1402124400, 'question': 'are these made by philips norelco?', 'answerType': 'Y', 'answer': 'Yes and they work great.each set last me one year.'}
{'questionType': 'open-ended', 'asin': '7293000154', 'answerTime': 'Apr 15, 2014', 'unixTime': 1397545200, 'question': 'I have a philips HQ 6695 does it fit?', 'answer': 'yes'}
{'questionType': 'open-ended', 'asin': '7293000154', 'answerTime': 'Mar 22, 2014', 'unixTime': 13954